<a href="https://colab.research.google.com/github/csaybar/EEwPython/blob/master/1_6_Map_filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!--COURSE_INFORMATION-->
<img align="left" style="padding-right:10px;" src="https://sitejerk.com/images/google-earth-logo-png-5.png" width=5% >
<img align="right" style="padding-left:10px;" src="https://colab.research.google.com/img/colab_favicon_256px.png" width=6% >


>> *This notebook is part from the free course [EEwPython](https://github.com/csaybar/EEwPython); the content is available [on GitHub](https://github.com/csaybar/EEwPython)* and released under the [Apache 2.0 License](https://www.gnu.org/licenses/gpl-3.0.en.html).

<!--NAVIGATION-->
 < [Conditional statements](1.5_Cond_Statements.ipynb) | [Contents](index.ipynb) |  [Spatial Temporal Reducers](1.7_Reducers.ipynb)>

<a href="https://colab.research.google.com/github/csaybar/EEwPython/blob/master/1.6_Map_filter.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

In [0]:
#@title # Filtering an Mapping over an ImageCollection

from IPython.display import HTML
HTML('<center><iframe width="560" height="315" src="https://www.youtube.com/embed/LzxQH0Ze0iI" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe></center>')

In [0]:
#@title Function to visualize thumbnail URL
from IPython.display import Image
from IPython.display import HTML

def display_image(url,title="",CSS_row=False):
  img_link = "<center> <h1>%s</h1> <img title='%s' src= '%s'></center>" % (title,title,url)
  return HTML(img_link)

### Connecting GEE, Colab and Google Drive

- **Colab & Google Drive synchronization**

In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

- **Colab & Earth Engine synchronization**

In [0]:
!pip install earthengine-api #Firstly we need the earth-engine API

In [0]:
!earthengine authenticate 

In [0]:
import ee
ee.Initialize()

### 1. Filtering an ImageCollection


Earth Engine provides a variety of convenience methods for filtering image collections. Specifically, many common use cases are handled by `imageCollection.filterDate()`, and `imageCollection.filterBounds()`. For general purpose filtering, use `imageCollection.filter()` with an `ee.Filter` as an argument. 

The following example demonstrates both convenience methods and `filter()` to identify and remove images with bad registration from an ImageCollection:

In [0]:
# Load Landsat 5 data, filter by date and bounds.
collection = ee.ImageCollection('LANDSAT/LT05/C01/T2')\
               .filterDate('1987-01-01', '1990-05-01')\
               .filterBounds(ee.Geometry.Point(25.8544, -18.08874))

# Also filter the collection by the IMAGE_QUALITY property.
filtered = collection.filterMetadata('IMAGE_QUALITY', 'equals', 9)

# Create two composites to check the effect of filtering by IMAGE_QUALITY.
badComposite = ee.Algorithms.Landsat.simpleComposite(collection, 75, 3)
goodComposite = ee.Algorithms.Landsat.simpleComposite(filtered, 75, 3)

# Display the composites.
viz_params = {'bands': ['B3', 'B2', 'B1'], 'gain': 3.5}
crop_image = ee.Geometry.Rectangle([25.8544-1, -18.08874-1,25.8544+1, -18.08874+1])

img1 = display_image(badComposite.clip(crop_image).getThumbURL(viz_params),'Bad Composite')
img2 = display_image(goodComposite.clip(crop_image).getThumbURL(viz_params),'Good Composite')

display(img1,img2)

### 2. Mapping over an ImageCollection

To apply a function to every `Image` in an `ImageCollection` use `imageCollection.map()`. The only argument to map() is a function which takes one parameter: an `ee.Image`. For example, the following code adds a timestamp band to every image in the collection:

In [0]:
# Load a Landsat 8 collection for a single path-row.
collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')\
               .filter(ee.Filter.eq('WRS_PATH', 44))\
               .filter(ee.Filter.eq('WRS_ROW', 34))

# This function adds a band representing the image timestamp.
def addTime(img):
  return img.addBands(img.metadata('system:time_start'))

# Map the function over the collection and display the result.
collection.map(addTime).getInfo()

Note that in the predefined function, the `metadata()` method is used to create a new `Image` from the value of a property.es.

The mapped function is limited in the operations it can perform. Specifically, it can’t modify variables outside the function; it can’t print anything; it can’t use Python ‘if’ or ‘for’ statements. However, you can use `ee.Algorithms.If()` to perform conditional operations in a mapped function. For example:

In [0]:
# Load a Landsat 8 collection for a single path-row.
collection = ee.ImageCollection('LANDSAT/LC8_L1T_TOA')\
               .filter(ee.Filter.eq('WRS_PATH', 44))\
               .filter(ee.Filter.eq('WRS_ROW', 34))

# This function uses a conditional statement to return the image if
# the solar elevation > 40 degrees.  Otherwise it returns a zero image.
def conditional(img):
  return ee.Algorithms.If(ee.Number(img.get('SUN_ELEVATION')).gt(40),
                          img,
                          ee.Image(0))

# Map the function over the collection, convert to a List and print the result.
print('Expand this to see the result: ')
collection.map(conditional).getInfo()

Inspect the list of images in the output ImageCollection and note that the when the condition evaluated by the `If()` algorithm is true, the output contains a constant image. Although this demonstrates a server-side conditional function (learn more about client vs. server in Earth Engine on [this page](https://developers.google.com/earth-engine/client_server)), avoid `If()` in general and use filters instead.



<!--NAVIGATION-->
 < [Conditional statements](1.5_Cond_Statements.ipynb) | [Contents](index.ipynb) |  [Spatial Temporal Reducers](1.7_Reducers.ipynb)>

<a href="https://colab.research.google.com/github/csaybar/EEwPython/blob/master/1.6_Map_filter.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>